The Merkle tree, ecdsa verification and Poseidon hash of python and circom are all come from: https://github.com/Sdoba16/Circom-DL-course/tree/main/MerkleTree <br>
Add subprocess calls, corresponding ecdsa generation and input.json generation to meet GitHub_ZKP. <br>
Note: Note: The ECDSA may not pass the check since the "contribute" step requires much more RAM, and I have not test it.<br>

In [2]:
import subprocess
import os
import sys

import json
import hashlib
from hashlib import sha3_256

from poseidon import Poseidon

from ecdsa import SigningKey, SECP256k1
from ecdsa.util import sigencode_strings

import time

# os.environ['PATH'] += r';C:\Users\SurfacePro\AppData\Roaming\npm\snarkjs'

def compile_proof():
    # circom github_proof_ecdsa.circom --r1cs --wasm --sym
    subprocess.run([
        "circom", "github_proof_ecdsa.circom", "--r1cs",
        "--wasm", "--sym"
    ], shell=True)
    
    # snarkjs groth16 setup github_proof_ecdsa.r1cs powersOfTau28_hez_final_21.ptau circuit_0000.zkey
    subprocess.run([
        "snarkjs", "groth16", "setup",
        "github_proof_ecdsa.r1cs", "powersOfTau28_hez_final_21.ptau",
        "circuit_0000.zkey"
    ], shell=True)
    
    print('setup done')
    
    # snarkjs zkey contribute circuit_0000.zkey circuit_0001.zkey
    contribute_cmd = [
        "snarkjs", "zkey", "contribute",
        "circuit_0000.zkey",
        "circuit_0001.zkey"
    ]
    
    entropy = b"zxcvbnM"
    proc = subprocess.Popen(
        contribute_cmd,
        stdin=subprocess.PIPE, shell=True
    )
    proc.communicate(input=entropy + b"\n\n")
    
    print('contribute done')
    
    subprocess.run([
        "snarkjs", "zkey", "export",
        "verificationkey", "circuit_0001.zkey",
        "verification_key.json"
    ], shell=True)
    
    print('export done')

def generate_and_verify_proof():
    # 1. Generate witness
    # node github_proof_ecdsa_js/generate_witness.js github_proof_ecdsa_js/github_proof_ecdsa.wasm input.json witness.wtns
    try:
        subprocess.run([
            "node", "github_proof_ecdsa_js/generate_witness.js",
            "github_proof_ecdsa_js/github_proof_ecdsa.wasm",
            "input.json", "witness.wtns"
        ], shell=True, capture_output=True, text=True, check=True)
    except subprocess.CalledProcessError as e:
        print("❌ FAIL: Witness generation failed (constraints not met)")
        #print(e.stderr.decode())
        return
    
    time.sleep(1)

    # 2. Generate proof
    # snarkjs groth16 prove circuit_0001.zkey witness.wtns proof.json public.json
    try:
        subprocess.run([
            "snarkjs", "groth16", "prove",
            "circuit_0001.zkey", "witness.wtns",
            "proof.json", "public.json"
        ], shell=True, capture_output=True, text=True, check=True)
    except subprocess.CalledProcessError as e:
        print("❌ FAIL: Proof generation failed")
        print(e.stderr.decode())
        return
    
    time.sleep(1)

    # 3. Verify proof
    # snarkjs groth16 verify verification_key.json public.json proof.json
    try:
        result = subprocess.run([
            "snarkjs", "groth16", "verify",
            "verification_key.json", "public.json", "proof.json"
        ], shell=True, capture_output=True, text=True, check=True)
    except subprocess.CalledProcessError as e:
        print("❌ FAIL: Verification failed")
        print(e.stderr.decode())
        return

    if "OK!" in result.stdout:
        print("✅ PASS: Proof is valid")
    else:
        print("❌ FAIL: Proof is invalid")
        print(result.stdout.decode())

In [3]:
# 1. Prepare inputs
length = 8
users = [{"username": f"user{i}", "stars": i*10 if i < length - 1 else 1000000} for i in range(length)]
target_index = 6
target_user = users[target_index]
threshold = 50

In [4]:
# Example message
def generate_circom_param(val):
    m = val.to_bytes(32, byteorder='big')

    # Step 1: Generate private/public key pair
    sk = SigningKey.generate(curve=SECP256k1)   # private key
    vk = sk.verifying_key                      # public key

    # Step 2: Hash the message
    digest = hashlib.sha256(m).digest()
    hui = int.from_bytes(digest, 'big')

    # Step 3: Sign the hashed message
    signature = sk.sign(hashlib.sha256(m).digest())

    # Step 4: Extract r and s from the signature
    r_bytes, s_bytes = sk.sign_digest_deterministic(
        digest,
        sigencode=sigencode_strings
    )

    # Convert to integers
    R = int.from_bytes(r_bytes, 'big')
    S = int.from_bytes(s_bytes, 'big')


    # Step 5: Get public key (x, y)
    pubkey_point = vk.pubkey.point
    pubkeyX = pubkey_point.x()
    pubkeyY = pubkey_point.y()
    
    return R, S, pubkeyX, pubkeyY, hui

R, S, pubkeyX, pubkeyY, hui = generate_circom_param(target_user['stars'])

def bigint_to_array(n, k, x):
    # Initialize mod to 1 (Python's int can handle arbitrarily large numbers)
    mod = 1
    for idx in range(n):
        mod *= 2

    # Initialize the return list
    ret = []
    x_temp = x
    for idx in range(k):
        # Append x_temp mod mod to the list
        ret.append(str(x_temp % mod))
        # Divide x_temp by mod for the next iteration
        x_temp //= mod  # Use integer division in Python

    return ret


def jsonify_and_save(R, S, pubkeyX, pubkeyY, hui, filename='input.json'):
    data = {
        "R": bigint_to_array(64, 4, R),
        "S": bigint_to_array(64, 4, S),
        "PubKey":    [bigint_to_array(64, 4, pubkeyX), bigint_to_array(64, 4, pubkeyY)],
        "msghash": bigint_to_array(64, 4, hui)
    }
    
    
    return data
#print(bigint_to_array(64, 32, signature),)
#print(bigint_to_array(64, 32, pubkey))
#print(bigint_to_array(64, 32, pubkey))
inputs = jsonify_and_save(R, S, pubkeyX, pubkeyY, hui)

In [5]:
# 2. Subclass MerkleTools to use Poseidon hash
class MerkleTree :
    def __init__(self, leafs) :
        self.leafs = leafs
        self.size = 1
        self.log = 1
        while self.size < len(leafs) :
            self.size *= 2
            self.log += 1     
        print(self.size)
        self.hashes = [0 for i in range(self.size * 2 - 1)]
        for i in range(self.size * 2 - 2, -1, -1) :
            if i > self.size - 2 :
                self.hashes[i] = Poseidon(1, [leafs[i - self.size + 1]]) if len(leafs) > i - self.size + 1 else Poseidon(1, [0])
            else : 
                self.hashes[i] = Poseidon(2, [self.hashes[i * 2 + 1], self.hashes[i * 2 + 2]])
        self.root = self.hashes[0]

    def genPath(self, leafPos) :
        path = MerklePath(self, leafPos)
        return path
    
    def checkPath(self, leaf, path, root) :
        return(path.checkPath(leaf, root))
               
class MerklePath :
    def __init__(self, Tree, leafPos) :
        self.path = [0 for i in range(Tree.log - 1)]
        self.order = [0 for i in range(Tree.log - 1)]
        index = leafPos + Tree.size - 1
        i = 0
        while(index != 0) :
            self.path[i] = Tree.hashes[index + 1 if index % 2 else index - 1]
            self.order[i] = index % 2
            index = (index - 1) // 2
            i += 1
    
    def checkPath(self, leaf, root) :
        hash = Poseidon(1, [leaf])
        print(hash)
        for i in range(len(self.path)) :
            print(self.path[i])
            if self.order[i] == 1:
                hash = Poseidon(2, [hash, self.path[i]])
                print(hash)
            else : 
                hash = Poseidon(2, [self.path[i], hash])
                print(hash)
        return hash == root


# 3. Build Merkle tree with Poseidon
# def pubkey_to_address(pubkey_x, pubkey_y):
#     # 64-byte uncompressed pubkey (no prefix 0x04)
#     pubkey_bytes = pubkey_x.to_bytes(32, 'big') + pubkey_y.to_bytes(32, 'big')
#     keccak_hash = sha3_256(pubkey_bytes).digest()
#     return int.from_bytes(keccak_hash[-20:], 'big')  # last 20 bytes = 160-bit address

# target_leaf = pubkey_to_address(pubkeyX, pubkeyY)
# leaves = [users[i]['stars'] if i != target_index else target_leaf for i in range(len(users))]
leaves = [user['stars'] for user in users]
# Initialize MerkleToolsPoseidon and add leaves
mt = MerkleTree(leaves)

print(mt.genPath(target_index).checkPath(leaves[target_index], mt.root))

inputs.update( {
    #"leaf": str(leaves[target_index]),  # The leaf is the target user's stars
    "root": str(mt.root),  # Merkle root in hex format
    
    "threshold": str(threshold),  # optional, depends on your full circuit
    "userStars": str(users[target_index]["stars"]),
    "pathElements": [str(x) for x in mt.genPath(target_index).path],  # Convert path elements to hex string
    "pathIndices": [str(x) for x in mt.genPath(target_index).order]  # Path indices (0 for left, 1 for right)
})

# 6. Generate proof input (write to JSON)
with open("input.json", "w") as f:
    json.dump(inputs, f)

# Note: cannot run following code due to OOM. cli will also cause OOM.
compile_proof()

generate_and_verify_proof()

8
1881250565452245586419841192852671026212652849919648586297501678806682883737
18537448666219337719725227014616684997453837229091063751344226035333080648533
6957687199911205590426370210959070230908924870386300963286684698616473644630
910383297561278391855733785252563526513054584183068095353007716894503418916
13465890954100483624472890896589001373524142036605766412850259057668510708330
20670160035248158214392568213822581776271255870317760628552874998276494994454
11278960386806594067589042694135511545202911877751106672063437668315164418171
True


In [6]:
Poseidon(2, [1881250565452245586419841192852671026212652849919648586297501678806682883737, 4015195612501362753079614620770870067126687168784179108455416718864718757888])

18262063480288803626647907806194038916518609110734112242713265424057789441715